In [ ]:
#| default_exp learner

In [ ]:
import pandas as pd, os

# Learner

## Imports

In [ ]:
#|export
import math,torch,matplotlib.pyplot as plt
import fastcore.all as fc
from collections.abc import Mapping
from operator import attrgetter
from functools import partial
from copy import copy

from torch import optim
import torch.nn.functional as F
from torch.utils.data import default_collate

# from miniai.conv import *

from fastprogress import progress_bar,master_bar
from operator import itemgetter

In [ ]:
import matplotlib as mpl
import torchvision.transforms.functional as TF
from contextlib import contextmanager
from torch import nn,tensor
from datasets import load_dataset,load_dataset_builder
import logging 

In [ ]:
from torch.utils.data.dataloader import DataLoader, Dataset

In [ ]:
logging.disable(logging.WARNING)

In [ ]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

## Dataset

In [ ]:
#| export
class DataLoaders:
    def __init__(self, *dls): self.train,self.valid = dls[:2]

    @classmethod
    def from_dd(cls, dd, batch_size, as_tuple=True, **kwargs):
        f = collate_dict(dd['train'])
        return cls(*get_dls(*dd.values(), bs=batch_size, collate_fn=f, **kwargs))

In [ ]:
x,y = 'image','label'
name = "fashion_mnist"
dsd = load_dataset(name)

In [ ]:
dsd["train"][x][0]

In [ ]:
#| export 
def inplace(f):
    def _f(b):
        f(b)
        return b
    return _f

In [ ]:
#| export 
def collate_dict(ds):
    get = itemgetter(*ds.features)
    def _f(b): return get(default_collate(b))
    return _f

In [ ]:
@inplace
def transformi(b): b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [ ]:
bs = 1024
tds = dsd.with_transform(transformi)

In [ ]:
def get_dls(train_ds, valid_ds, bs, **kwargs):
    return (DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs),
            DataLoader(valid_ds, batch_size=bs*2, **kwargs))

In [ ]:
dls = DataLoaders.from_dd(tds, bs, num_workers=4)
dt = dls.train


In [ ]:
dt.collate_fn

In [ ]:
xb,yb = next(iter(dt))
xb.shape,yb[:10]

## Callback 

In [ ]:
#| export
class cb:
    order = 0
        

In [ ]:
class testcb(cb):
    def __init__(self):fc.store_attr()
        
    def before_fit(self, learn):
        print(f"starting fit function")
        
    def after_fit(self, learn):
        print(f"ending fit function with loss : {learn.loss}")

In [ ]:
# class testcb1(cb):
#     def __init__(self):
#         fc.store_attr()
#         self.order = -1
        
#     def before_fit(self, learn):
#         print(f"starting fit function")
        
#     def after_fit(self, learn):
#         print(f"ending fit function with loss : {learn.loss}")

In [ ]:
cbs = [ testcb()]

In [ ]:
sorted(cbs, key= attrgetter("order"))

In [ ]:
if getattr(testcb(), "before_fit"): pass

In [ ]:
#| export

def rcb(cbs, method_name, learn):
    for cb in sorted(cbs, key= attrgetter("order")):
        method =  getattr(cb, method_name, None )
        if method : return method(learn)
        

## Learner Framework Basic

In [ ]:
#| export 
def_device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'

def to_device(x, device=def_device):
    if isinstance(x, torch.Tensor): return x.to(device)
    if isinstance(x, Mapping): return {k:v.to(device) for k,v in x.items()}
    return type(x)(to_device(o, device) for o in x)

In [ ]:

class learner:
    
    def __init__(self, model, dls,loss_func, lr,opt_func=optim.SGD):fc.store_attr()
    
    def one_batch(self):
        x,y = to_device(self.b)
        self.preds = self.model(x)
        self.loss = self.loss_func(self.preds, y)
        if self.model.training: 
            self.loss.backward()
            self.opt.step()
            self.opt.zero_grad()
#         else :
#             with torch.no_grad():
#                 self.preds = self.model(b[0])
            
        
    
    def one_epoch(self, train):
        self.model.training = train
        dl = self.dls.train if train else self.dls.valid
        for self.b in dl:self.one_batch()
        
    def fit(self, epochs):
        self.model = self.model.to(def_device)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        for e in range(epochs):
            self.one_epoch(True)
            with torch.no_grad(): self.one_epoch(False)
                
            print(f"epoch {e} loss :{self.loss}")
        
    

In [ ]:
m,nh = 28*28,50
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))

In [ ]:
learn = learner(model, dls, F.cross_entropy, lr=0.2)
learn.fit(3)

## Flexible Learner

In [ ]:
class ctx_mng():
    def __init__(self, name):self.name = name
        
    def __enter__(self):print(f"Executing code before {self.name} function")
        
    def __exit__(self, *args,**kwargs):
        print(f"Executing code after{self.name} function")
   
    

    

In [ ]:
with ctx_mng("before") as ctx:
    print("function")

In [ ]:
class learner:
    
    def __init__(self, model, dls,loss_func, lr,cbs ,opt_func=optim.SGD):fc.store_attr()
    
    def one_batch(self):
        x,y = to_device(self.b)
        with ctx_mng('batch'):
            self.preds = self.model(x)
            self.loss = self.loss_func(self.preds, y)
            if self.model.training: 
                self.loss.backward()
                self.opt.step()
                self.opt.zero_grad()
#         else :
#             with torch.no_grad():
#                 self.preds = self.model(b[0])
            
        
    
    def one_epoch(self, train):
        self.model.training = train
        dl = self.dls.train if train else self.dls.valid
        with ctx_mng('epoch'):
            for self.b in dl:self.one_batch()
        
    def fit(self, epochs):
        self.model = self.model.to(def_device)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        with ctx_mng('fit'):
            for e in range(epochs):
                self.one_epoch(True)
                with torch.no_grad(): self.one_epoch(False)

                print(f"epoch {e} loss :{self.loss}")

In [ ]:
m,nh = 28*28,50
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))

In [ ]:
learn = learner(model, dls, F.cross_entropy, lr=0.2, cbs = [])
learn.fit(1)

## More Flexible learner 

In [ ]:
#| export

class CancelFitException:pass
class CancelEpochException:pass
class CancelBatchException:pass

In [ ]:
"fit".title()

In [ ]:
#| export
class cb_dec:
    def __init__(self, name):
        fc.store_attr()
        
    def __call__(self, f):
        def _f(o,*args,**kwargs):
            try:
                o.callback(f"before_{self.name}")
                f(o,*args, **kwargs)
                o.callback(f"after_{self.name}")
            except: globals()[f"Cancel{self.name.title()}Exception"]()
        return _f 
            
            

In [ ]:
#| export 
class learner:
    
    def __init__(self, model, dls,loss_func, lr,cbs=[] ,opt_func=optim.SGD):fc.store_attr()
    
    @cb_dec("batch")
    def one_batch(self):
        x,y = to_device(self.b)
        
        self.preds = self.model(x)
        self.loss = self.loss_func(self.preds, y)
        if self.model.training: 
            self.loss.backward()
            self.opt.step()
            self.opt.zero_grad()
#         else :
#             with torch.no_grad():
#                 self.preds = self.model(b[0])
            
        
    @cb_dec("epoch")
    def one_epoch(self, train):
        self.model.training = train
        dl = self.dls.train if train else self.dls.valid
        
        for self.b in dl:self.one_batch()
    
    @cb_dec("fit")
    def fit(self, epochs):
        self.model = self.model.to(def_device)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        
        for e in range(epochs):
            self.one_epoch(True)
            with torch.no_grad(): self.one_epoch(False)

            print(f"epoch {e} loss :{self.loss}")
                
    def callback(self, nm): rcb(self.cbs, nm, self)
        

In [ ]:
# def rcb(cbs, method_name, learn):
#     for cb in sorted(cbs, key= attrgetter("order")):
#         try: return getattr(cb, method_name)(learn)
#         except Exception as e: print(e)

In [ ]:
m,nh = 28*28,50
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))

In [ ]:
learn = learner(model, dls, F.cross_entropy,lr=0.2,cbs = [testcb()])
learn.fit(1)

## Trainer Learner

In [ ]:
#|export
class TrainLearner(learner):
    def predict(self): self.preds = self.model(self.batch[0])
    def get_loss(self): self.loss = self.loss_func(self.preds, self.batch[1])
    def backward(self): self.loss.backward()
    def step(self): self.opt.step()
    def zero_grad(self): self.opt.zero_grad()

In [ ]:
m,nh = 28*28,50
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))

learn = TrainLearner(model, dls, F.cross_entropy,lr=0.2,cbs = [testcb()])
learn.fit(1)

## Momentum Learner


In [ ]:
#| export 
class momentumLearner(learner):
    def __init__(self, model, dls, loss_func, lr=None, cbs=None, opt_func=optim.SGD, mom=0.85):
        self.mom = mom
        super().__init__(model, dls, loss_func, lr, cbs, opt_func)
        
    def step(self): self.opt.step()
        
    def zero_grad(self): 
        with torch.no_grad():
            for p in self.model.parameters(): p.grad *= self.mom

In [ ]:
m,nh = 28*28,50
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))

learn = TrainLearner(model, dls, F.cross_entropy,lr=0.2,cbs = [testcb()])
learn.fit(1)

## Metrics Callback

In [ ]:
from torcheval.metrics import MulticlassAccuracy

In [ ]:
ms = [MulticlassAccuracy()]

In [ ]:
class Metrics(cb):
    ...




In [ ]:
metricss={}
metricss[type(ms[0]).__name__] = ms[0]

In [ ]:
metricss

In [ ]:
#| hide 
import nbdev; nbdev.nbdev_export()